<h1>Análise de partidas de Padel</h1>

<h3>Primeiro importar o dataset</h3>

In [ ]:
import pandas as pd

df = pd.read_csv ('drive/MyDrive/Colab Notebooks/trabalho_1_mineracao/_ASSOC_PadelStars.csv')
df

,Partida,Jogadore(a)s,Resultado,Unnamed: 3,Jogadore(a)s.1
0,1,"Carlos, alejandro, Alonso",Perdeu,NaN,Alejandro
1,2,"Alonso, alejandro, Juan",Perdeu,NaN,Ariana
2,3,"Juan, Carlos, alejandro",GANHOU,NaN,Carlos
3,4,"Juan, alejandro, Paula",GANHOU,NaN,Fernando
4,5,"Paula, Ariana, Lucia",Perdeu,NaN,Juan
...,...,...,...,...,...
146,147,"Fernando, Lucia",Perdeu,NaN,NaN
147,148,"Juan, Carlos, Alonso",GANHOU,NaN,NaN
148,149,"Juan, alejandro, Alonso.",Perdeu,NaN,NaN
149,150,"alejandro, Paula.",GANHOU,NaN,NaN


<h3>Remover coluna nula, arrumar nome das outras colunas, colocar em lower case e remover todos espaços em branco</h3>

In [ ]:
df = df.drop(['Unnamed: 3', 'Partida', 'Jogadore(a)s.1'], 1) #remover coluna nula e colunas não importantes

In [ ]:
df = df.rename(columns={'Jogadore(a)s': 'Jogadores'}) #renomear coluna

In [ ]:
df['Jogadores'] = df['Jogadores'].str.lower() #colocar lowercase

In [ ]:
df['Jogadores'] = df['Jogadores'].str.replace(' ', '') #retirar whitespaces

In [ ]:
df['Jogadores'] = df['Jogadores'].str.replace('*', '') #retirar caracter especial

In [ ]:
df['Jogadores']= df['Jogadores'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') #retirar acentos

In [ ]:
df['Jogadores'] = df['Jogadores'].str.replace('juam', 'juan') #trocar juam por juan

In [ ]:
df['Jogadores'] = df['Jogadores'].str.replace(',alonso', '') #retirar o alonso
df['Jogadores'] = df['Jogadores'].str.replace('alonso,', '') #retirar o alonso
df['Jogadores'] = df['Jogadores'].str.replace('alonso', '') #retirar o alonso

<h3>Separar os nomes de uma coluna em várias colunas, ou seja, cada nome será um atributo no Dataframe</h3>

In [ ]:
df_nomes_separados = pd.concat([df, df['Jogadores'].str.split(',', expand=True)], axis=1)
df_nomes_separados = df_nomes_separados.drop('Jogadores', 1)

In [ ]:
df_nomes_separados = df_nomes_separados.rename(columns={0: 'Jogador1', 1: 'Jogador2', 2: 'Jogador3'})
df_nomes_separados['Jogador1'] = df_nomes_separados['Jogador1'].str.replace(r'.', '')
df_nomes_separados['Jogador2'] = df_nomes_separados['Jogador2'].str.replace(r'.', '')
df_nomes_separados['Jogador3'] = df_nomes_separados['Jogador3'].str.replace(r'.', '')
df_nomes_separados

,Resultado,Jogador1,Jogador2,Jogador3
0,Perdeu,carlos,alejandro,None
1,Perdeu,alejandro,juan,None
2,GANHOU,juan,carlos,alejandro
3,GANHOU,juan,alejandro,paula
4,Perdeu,paula,ariana,lucia
...,...,...,...,...
146,Perdeu,fernando,lucia,None
147,GANHOU,juan,carlos,None
148,Perdeu,juan,alejandro,None
149,GANHOU,alejandro,paula,None


<h3>Fazer OneHot Encode com os nomes e resultado</h3>

In [ ]:
df_jogadores = df_nomes_separados[['Jogador1', 'Jogador2', 'Jogador3', 'Resultado']]
df_onehot = df_jogadores.stack().str.get_dummies().sum(level=0)
df_onehot

,GANHOU,Perdeu,alejandro,ariana,carlos,fernando,juan,lucia,paula
0,0,1,1,0,1,0,0,0,0
1,0,1,1,0,0,0,1,0,0
2,1,0,1,0,1,0,1,0,0
3,1,0,1,0,0,0,1,0,1
4,0,1,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...
146,0,1,0,0,0,1,0,1,0
147,1,0,0,0,1,0,1,0,0
148,0,1,1,0,0,0,1,0,0
149,1,0,1,0,0,0,0,0,1


<h3>TESTE 2</h3>

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

class Apriori:
    """Classe Apriori."""
    suporte = 0.5
    df = None

    def __init__(self, df, suporte=None, transform_bol=False):
        """Construtor. 

        :param pandas.DataFrame df: dataframe OneHot Encode.
        :param float suporte: suporte para min_support.
        :return: Instância Apriori.
        :rtype: Apriori
        """

        self._validar_df(df)

        self.df = df
        if suporte is not None:
            self.suporte = suporte

        if transform_bol:
            self._transform_bol()

    def _validar_df(self, df=None):
        """Validade se df existe. 

        :param pandas.DataFrame df: dataframe OneHot Encode.
        :return: 
        :rtype: void
        """

        if df is None:
            raise Exception("df deve ser um pandas.DataDrame válido.")


    def _transform_bol(self):
        """Transformar dataframe OneHot Encode para (True ou False). 

        :return: 
        :rtype: void
        """

        for coluna in self.df.columns:
            self.df[coluna] = self.df[coluna].apply(lambda x: True if x == 1 else False)


    def _apriori(self, use_colnames=False, max_len=None, count=True):
        """Chama a função apriori do mlxtend.frequent_patterns. 

        :param bool use_colnames: Flag para usar o nome das colunas no dataframe.
        :param int max_len: Tamanho máximo de itemsets gerados.
        :param bool count: Flag para contar o número de itemsets.
        :return: apriori DataFrame.
        :rtype: pandas.DataFrame
        """
    
        apriori_df = apriori(
                    self.df, 
                    min_support=self.suporte,
                    use_colnames=use_colnames, 
                    max_len=max_len
                )

        if count:
            apriori_df['length'] = apriori_df['itemsets'].apply(lambda x: len(x))

        return apriori_df

    def run(self, use_colnames=False, max_len=None, count=True):
        """ Execução Apriori.

        :param bool use_colnames: Flag para usar o nome das colunas no dataframe.
        :param int max_len: Tamanho máximo de itemsets gerados.
        :param bool count: Flag para contar o número de itemsets.
        :return: apriori DataFrame.
        :rtype: pandas.DataFrame
        """

        return self._apriori(
                        use_colnames=use_colnames,
                        max_len=max_len,
                        count=count
                    )

    def filter(self, apriori_df, tamanho, suporte):
        """Filtrar o dataframe Apriori por tamanho e suporte.

        :param pandas.DataFrame apriori_df: Apriori DataFrame.
        :param int length: Length of itemsets required.
        :param float threshold: Minimum threshold nrequired.
        :return: apriori filtered DataFrame.
        :rtype:pandas.DataFrame
        """
        
        if 'length' not in apriori_df.columns:
            raise Exception("apriori_df não tem atributo tamanho. Por favor execute o Apriori com count=True.")

        return apriori_df[ (apriori_df['length'] >= tamanho) & (apriori_df['support'] >= suporte) ]

    def association_rules(self, apriori_df, suporte):
      return association_rules(apriori_df, metric="confidence", min_threshold=suporte)

In [ ]:
df_apriori =  df_onehot
apriori_runner = Apriori(df_apriori, suporte=0.1, transform_bol=True)
df_apriori = apriori_runner.run(use_colnames=True)
pd.set_option("display.max_rows", None, "display.max_columns", None)
df_apriori

,support,itemsets,length
0,0.549669,(GANHOU),1
1,0.450331,(Perdeu),1
2,0.430464,(alejandro),1
3,0.443709,(ariana),1
4,0.278146,(carlos),1
5,0.205298,(fernando),1
6,0.417219,(juan),1
7,0.264901,(lucia),1
8,0.417219,(paula),1
9,0.284768,"(GANHOU, alejandro)",2


In [ ]:
# apriori_runner.filter(df_apriori, tamanho=2, suporte=0.1)

,support,itemsets,length
9,0.284768,"(GANHOU, alejandro)",2
10,0.251656,"(ariana, GANHOU)",2
11,0.145695,"(carlos, GANHOU)",2
12,0.245033,"(juan, GANHOU)",2
13,0.125828,"(lucia, GANHOU)",2
14,0.238411,"(GANHOU, paula)",2
15,0.145695,"(alejandro, Perdeu)",2
16,0.192053,"(ariana, Perdeu)",2
17,0.132450,"(carlos, Perdeu)",2
18,0.132450,"(fernando, Perdeu)",2


<h5>Melhor jogador</h5>

In [ ]:
df_melhores_jogadores = apriori_runner.association_rules(df_apriori, suporte=0.1)
condicao_1 = df_melhores_jogadores['antecedents'].apply(lambda x: len(x) == 1)
condicao_2 = df_melhores_jogadores['consequents'].apply(lambda x: 'GANHOU' in x and len(x) == 1)
df_melhores_jogadores = df_melhores_jogadores.loc[condicao_1 & condicao_2]
df_melhores_jogadores

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(alejandro),(GANHOU),0.430464,0.549669,0.284768,0.661538,1.203522,0.048156,1.330524
2,(ariana),(GANHOU),0.443709,0.549669,0.251656,0.567164,1.031829,0.007763,1.040420
4,(carlos),(GANHOU),0.278146,0.549669,0.145695,0.523810,0.952955,-0.007193,0.945695
6,(juan),(GANHOU),0.417219,0.549669,0.245033,0.587302,1.068464,0.015701,1.091187
8,(lucia),(GANHOU),0.264901,0.549669,0.125828,0.475000,0.864157,-0.019780,0.857774
11,(paula),(GANHOU),0.417219,0.549669,0.238411,0.571429,1.039587,0.009079,1.050773


<h5>Pior jogador</h5>

In [ ]:
df_piores_jogadores = apriori_runner.association_rules(df_apriori, suporte=0.1)
condicao_1 = df_piores_jogadores['antecedents'].apply(lambda x: len(x) == 1)
condicao_2 = df_piores_jogadores['consequents'].apply(lambda x: 'Perdeu' in x and len(x) == 1)
df_piores_jogadores = df_piores_jogadores.loc[condicao_1 & condicao_2]
df_piores_jogadores

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
12,(alejandro),(Perdeu),0.430464,0.450331,0.145695,0.338462,0.751584,-0.048156,0.830895
14,(ariana),(Perdeu),0.443709,0.450331,0.192053,0.432836,0.961150,-0.007763,0.969153
16,(carlos),(Perdeu),0.278146,0.450331,0.132450,0.476190,1.057423,0.007193,1.049368
18,(fernando),(Perdeu),0.205298,0.450331,0.132450,0.645161,1.432638,0.039998,1.549067
20,(juan),(Perdeu),0.417219,0.450331,0.172185,0.412698,0.916433,-0.015701,0.935923
22,(lucia),(Perdeu),0.264901,0.450331,0.139073,0.525000,1.165809,0.019780,1.157198
24,(paula),(Perdeu),0.417219,0.450331,0.178808,0.428571,0.951681,-0.009079,0.961921


<h5>Melhor combinação</h5>

In [ ]:
df_combinacao = apriori_runner.association_rules(df_apriori, suporte=0.1)
condicao_1 = df_combinacao['antecedents'].apply(lambda x: len(x) >= 2)
condicao_2 = df_combinacao['consequents'].apply(lambda x: 'GANHOU' in x and len(x) == 1)
df_combinacao = df_combinacao.loc[condicao_1 & condicao_2]
df_combinacao

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
37,"(juan, alejandro)",(GANHOU),0.178808,0.549669,0.119205,0.666667,1.212851,0.020920,1.350993
43,"(paula, alejandro)",(GANHOU),0.165563,0.549669,0.119205,0.720000,1.309880,0.028201,1.608325
49,"(ariana, paula)",(GANHOU),0.231788,0.549669,0.132450,0.571429,1.039587,0.005044,1.050773
